# Поиск наиболее перспективного региона для разработки нефтяных месторождений 
______

#### Данное исследование разделим на несколько частей.
______

### Часть 1. Изучение общей информации, подготовка данных:
* [1. Изучение файлов с данными, получение общей информации, загрузка библиотек.](#1-bullet)
* [2. Подготовка данных.](#2-bullet)

### Часть 2. Обучение и проверка моделей:
* [1. Разделение данных на обучающую и валидационную выборки.](#3-bullet)
* [2. Обучение моделей и получение предсказаний.](#4-bullet)
* [3. Сохранение результатов и выводы о работе моделей.](#5-bullet)

### Часть 3. Подготовка к расчету прибыли:
* [1. Создание констант для всех ключевых значений.](#6-bullet)
* [2. Расчет минимального среднего количества продукта в месторождениях региона, достаточного для разработки.](#7-bullet)

### Часть 4. Расчет рисков и прибыли:
* [1. Применение метода Bootstrap.](#8-bullet)
* [2. Поиск наиболее перспективного региона для разработки месторождений.](#9-bullet)

#### Описание данных:
Данные геологоразведки трёх регионов находятся в файлах:

- geo_data_0.csv.
- geo_data_1.csv.
- geo_data_2.csv.

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Стоимость одной тыс. баррелей = 450т.р

#### 1-bullet
### Часть 1. Изучение общей информации, подготовка данных:
#### 1. Изучение файлов с данными, получение общей информации, загрузка библиотек.

загружаем все необходимые библиотеки.

In [1]:
# загружаем все необходимые для данного исследования библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

открываем файлы и получаем общую информацию.

In [2]:
# открываем файлы с данными 
# и сохраняем в переменных
region_1 = pd.read_csv('/datasets/geo_data_0.csv')
region_2 = pd.read_csv('/datasets/geo_data_1.csv')
region_3 = pd.read_csv('/datasets/geo_data_2.csv')

# создаем список объектов
regions = [region_1, region_2, region_3]

# создаем цикл для каждого региона
for i in regions:
    # получаем общую информацию по таблицам
    display(i.info())
    # выведем на экран сумму дубликатов в таблицах
    display(i.duplicated().sum()) 
    # выводим на экран первые пять строк для 
    # зрительного ознакомления
    display(i.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

0

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


### Вывод

- загрузили необходимые библиотеки.
- открыли исходные файлы и сохранили в переменных.
- создали цикл для получения общих данных по всем таблицам.
- все таблицы идентичные: по 5 столбцов и 100000 строк, не содержат пропусков и дубликатов, содержат два типа данных.

#### 2-bullet
### Часть 1. Изучение общей информации, подготовка данных:
#### 2. Подготовка данных.

посмотрим на корреляцию признаков между собой в каждой таблице.

In [3]:
# создадим цикл для перебора столбцов
for i in region_1.drop(['id'], axis=1):
    for j in region_1.drop(['id'], axis=1):
        # при помощи метода corr() выведем корреляцию признаков
        if i != j and 0.85 < region_1[i].corr(region_1[j]) or region_1[i].corr(region_1[j]) < -0.85:
            print(f'{i} : {j} корреляция = {region_1[i].corr(region_1[j])}')
            print()

In [4]:
# создадим цикл для перебора столбцов
for i in region_2.drop(['id'], axis=1):
    for j in region_2.drop(['id'], axis=1):
        # при помощи метода corr() выведем корреляцию признаков
        if i != j and 0.85 < region_1[i].corr(region_2[j]) or region_2[i].corr(region_2[j]) < -0.85:
            print(f'{i} : {j} корреляция = {region_2[i].corr(region_2[j])}')
            print()

In [5]:
# создадим цикл для перебора столбцов
for i in region_3.drop(['id'], axis=1):
    for j in region_3.drop(['id'], axis=1):
        # при помощи метода corr() выведем корреляцию признаков
        if i != j and 0.85 < region_3[i].corr(region_3[j]) or region_3[i].corr(region_3[j]) < -0.85:
            print(f'{i} : {j} корреляция = {region_3[i].corr(region_3[j])}')
            print()

- пришли к выводу, что сильной корреляции в между признаками ни в одной таблице не имеется.

#### 3-bullet
### Часть 2. Обучение и проверка моделей:
#### 1. Разделение данных на обучающую и валидационную выборки.

сначала получим признаки и целевой признак.

In [6]:
# получим признаки
features_r1 = region_1.drop(['product', 'id'], axis=1)
features_r2 = region_2.drop(['product', 'id'], axis=1)
features_r3 = region_3.drop(['product', 'id'], axis=1)

# получим целевые признаки
target_r1 = region_1['product']
target_r2 = region_2['product']
target_r3 = region_3['product']

далее разобъем признаки и целевой признак на обучающую и валидационную выборки.

In [7]:
# создадим список составных частей имен переменных
regions_name = ['r1', 'r2', 'r3']

# сохраним значение параметра random_state в переменной 
my_random_state = 123

# создадим цикл чтобы разделить выборки во всех таблицах
for j in regions_name:
    (globals()['features_train_%s' % j], globals()['features_valid_%s' % j], 
     globals()['target_train_%s' % j], globals()['target_valid_%s' % j]) = (
                                train_test_split(globals()['features_%s' % j], 
                                globals()['target_%s' % j],
                                test_size=0.25, random_state=my_random_state))

### Вывод

- во всех таблицах разделили данные на обучающую и валидационную выборки в соотношении 75:25.

#### 4-bullet
### Часть 2. Обучение и проверка моделей:
#### 2. Обучение моделей и получение предсказаний.

сначала создадим функцию, которая будет обучать модель и получать предсказания, одновременно приводя предсказания к нужному виду.

In [8]:
# создадим функцию, чтобы получить предсказания на основе модели
# линейной регрессии и имеющихся выборок
def model_predict (features_train, target_train, 
                   features_valid, target_valid):
    # построим модель и обучим
    model = LinearRegression()
    model.fit(features_train, target_train)
    # получим предсказания и сделаем их объектом Series
    # и сохраним индексацию целевого признака для поиска соответствий
    predictions_valid = model.predict(features_valid)
    predictions_valid = pd.Series(predictions_valid, 
                                  index = target_valid.index) 
    return predictions_valid

получим предсказания по 1 региону и сохраним в переменной.

In [9]:
# применим функцию и сохраним предсказания в регионе 1
predictions_valid_r1 = model_predict(features_train_r1, 
                                     target_train_r1, 
                                     features_valid_r1, 
                                     target_valid_r1)
# выведем на экран
display(predictions_valid_r1)

42083    123.284256
71825     75.838240
99535     55.526760
47879     86.673045
36734    109.909327
            ...    
56405     89.708779
50954     55.672078
99121    139.042911
4170      96.867927
18144     54.714331
Length: 25000, dtype: float64

далее получим предсказания по 2 региону и сохраним в переменной.

In [10]:
# применим функцию и сохраним предсказания в регионе 2
predictions_valid_r2 = model_predict(features_train_r2, 
                                     target_train_r2, 
                                     features_valid_r2, 
                                     target_valid_r2)
# выведем на экран
display(predictions_valid_r2)

42083     56.056314
71825     81.589811
99535     54.461926
47879    108.660979
36734     54.479416
            ...    
56405     28.805734
50954    110.025164
99121      3.278982
4170      53.705839
18144    135.021879
Length: 25000, dtype: float64

далее получим предсказания по 3 региону и сохраним в переменной.

In [11]:
# применим функцию и сохраним предсказания в регионе 3
predictions_valid_r3 = model_predict(features_train_r3, 
                                     target_train_r3, 
                                     features_valid_r3, 
                                     target_valid_r3)
# выведем на экран
display(predictions_valid_r3)

42083     94.075161
71825    122.604414
99535     80.055039
47879    100.175906
36734     89.804044
            ...    
56405     92.274315
50954     93.406866
99121     67.803679
4170      84.156844
18144    127.484202
Length: 25000, dtype: float64

### Вывод

- на основе модели линейной регрессии получили предсказания по валидационной выборке и сохранили в переменных, одновременно придав предсказаниям такой же вид, как у целевого признака на валидационной выборке.

#### 5-bullet
### Часть 2. Обучение и проверка моделей:
#### 3. Сохранение результатов и выводы о работе моделей.

создадим функцию для расчета RMSE моделей и подсчета среднего запаса.

In [12]:
# создадим функцию для расчета RMSE и среднего предсказаний
# на основе целевого признака на вал выборке и предсказаний
def rmse_model(target_valid, predictions_valid):
    result = mean_squared_error(target_valid, 
                                predictions_valid) ** 0.5
    mean_predict = predictions_valid.mean()
    
    print('RMSE модели', j, result)
    print('Средний запас', j, mean_predict)

получим искомые значения, применив функцию.

In [13]:
# создадим цикл для необходимых переменных
# и применим функцию
for j in regions_name:
    rmse_model(globals()['target_valid_%s' % j], 
               globals()['predictions_valid_%s' % j])  

RMSE модели r1 37.64786282376176
Средний запас r1 92.54936189116309
RMSE модели r2 0.8954139804944304
Средний запас r2 69.28001860653976
RMSE модели r3 40.12803006598514
Средний запас r3 95.09859933591373


### Вывод

- получили наименьший показатель RMSE по данным региона 2, т.к. данные не сильно отличаются от среднего.
- большой разброс данных (почти одинаковый) в регионах 1 и 3 и как следствие большой показатель RMSE, вместе с ним можно предположить, что и риски выше.

#### 6-bullet
### Часть 3. Подготовка к расчету прибыли:
#### 1. Создание констант для всех ключевых значений.

In [14]:
# сохранили переменную суммы бюджета на разработку
budjet = 10000000000
# сохранили переменную стоимости 1 тыс. баррелей.
income_barrel = 450000
# сохранили переменную количества скважин для разведки.
research_well = 500
# сохранили переменную количества лучших скважин.
best_well = 200

сохранили все необходимые ключевые переменные.

#### 7-bullet
### Часть 3. Подготовка к расчету прибыли:
#### 2. Расчет минимального среднего количества продукта в месторождениях региона, достаточного для разработки.

найдем минимальный объем сырья в скважине с учетом бюджета и количества скважин, которые будут разрабатываться.

In [15]:
# найдем минимальный объем сырья для безубыточности 
# при разработке 200 скважин
min_prod = budjet / income_barrel / best_well
print('Минимальный объем сырья в скважине для безубыточности', 
      min_prod, 'т.')

Минимальный объем сырья в скважине для безубыточности 111.11111111111111 т.


- как видим, ранее рассчитанные средние запасы сырья во всех регионах ниже минимального объема сырья для безубыточной разработки, разработка скважин наугад без оценки рисков невозможна.

С учетом минимального объема сырья для безубыточности, найдем долю ликвидных и неликвидных скважин. Для этого сначала создадим функцию, которая создаст новый признак, где ликвидные скважины примут значение 1, а неликвидные - 0.

In [16]:
# создадим функцию для каждого региона
# которая будет 
def prof_loss_well(regions):
    for i in regions:    
        # выделим ликвидные и убыточные
        i['profit'] = np.where(i['product'] > min_prod, 1, 0)
        # выведем на экран доли
        print('Доля ликвидных скважин', 
              i.query('profit == 1')['profit'].value_counts()[1] 
              / len(i))
        print('Доля неликвидных скважин', 
              i.query('profit == 0')['profit'].value_counts()[0] 
              / len(i))

In [17]:
# применим функцию
prof_loss_well(regions)

Доля ликвидных скважин 0.36583
Доля неликвидных скважин 0.63417
Доля ликвидных скважин 0.16537
Доля неликвидных скважин 0.83463
Доля ликвидных скважин 0.38178
Доля неликвидных скважин 0.61822


- как видим во всех регионах доля ликвидных скважин невелика. в первом регионе - 36 %, во втором регионе - 16%, в третьем регионе 38%.

создадим функцию для расчета прибыли 200 лучших скважин на основе предсказаний, но по соответствующим им реальным значениям.

In [18]:
# объявим функцию, которая будет принимать на вход
# целевой признак, предсказания и число скважин
def revenue(target, probab, count):
    # отсортируем предсказания по убыванию
    probs_sorted = probab.sort_values(ascending=False)
    # по индексам отсорт-х значений предсказаний
    # найдем реальные значения
    selected = target[probs_sorted.index][:best_well]
    # рассчитаем прибыль
    return income_barrel * selected.sum() - budjet

### Вывод

- получили минимальный объем сырья в скважине для безубыточности - 111.11 т. при этом  ранее рассчитанные средние запасы сырья во всех регионах ниже указанного объема, ввиду чего разработка скважин наугад без оценки рисков невозможна.
- также пришли к выводу, что во всех регионах доля ликвидных скважин невелика. в первом регионе - 36 %, во втором регионе - 16%, в третьем регионе 38%. 

#### 8-bullet
### Часть 4. Расчет рисков и прибыли:
#### 1. Применение метода Bootstrap.

### Вывод

In [19]:
# создадим цикл для перебора имен переменных
for j in regions_name:
    target = globals()['target_valid_%s' % j]
    probab = globals()['predictions_valid_%s' % j]

    # сохраним параметр random_state
    state = np.random.RandomState(12345)
    # создадим список
    values = []
    # создадим цикл для 1000 выборок
    for i in range(1000):
        # создадим 500 случайных подвыборок
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        # найдем по индексам соответсвия реальных значений предсказаниям
        probs_subsample = probab[target_subsample.index]
        # применим фукнцию поиска 200 лучших скважин из выборки в 500
        # и рассчитаем средний доход
        res = revenue(target_subsample, probs_subsample, 200)
        # сохраним значения в списке
        values.append(res)
       
    # список приведем к объекту Series
    values = pd.Series(values)
    # найдем 95% доверительный интервал,
    # среднее и выведем на экран
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    print(j, 'cредняя выручка: {:.1e}'.format(mean), 'млд.рублей')
    print(j, '2.5%-квантиль: {:.2f}'.format(lower/budjet), 'млд.рублей')
    print(j, '97.5%-квантиль: {:.2f}'.format(upper/budjet), 'млд.рублей')
    print(j, 'риск убытков {0:.1%}'.format(len(values[values < 0])/len(values)))

r1 cредняя выручка: 5.1e+08 млд.рублей
r1 2.5%-квантиль: -0.00 млд.рублей
r1 97.5%-квантиль: 0.11 млд.рублей
r1 риск убытков 2.6%
r2 cредняя выручка: 5.4e+08 млд.рублей
r2 2.5%-квантиль: 0.01 млд.рублей
r2 97.5%-квантиль: 0.10 млд.рублей
r2 риск убытков 0.6%
r3 cредняя выручка: 3.8e+08 млд.рублей
r3 2.5%-квантиль: -0.02 млд.рублей
r3 97.5%-квантиль: 0.09 млд.рублей
r3 риск убытков 9.4%


#### 9-bullet
### Часть 4. Расчет рисков и прибыли:
#### 2. Поиск наиболее перспективного региона для разработки месторождений.

### Вывод

получили следующие результаты.

- по средней выручке: наибольшая средняя выручка во втором регионе - 5.4 млрд., в первом - 5.1 млрд. и в третьем - 3.8 млрд.
- по рискам убытков: наименьший риск убытков во втором регионе - 0.6%, затем в первом регионе - 2.6% и самый большой риск убытков в третьем регионе - 9.4%.
- также для каждого региона был найдет 95% доверительный интервал.

Таким образом, по результатам исследования второй регион является предпочтительным для разработки месторождений нефти с точки зрения прогноза наибольшей средней прибыли и наименьших рисков убытоков.